In [12]:
import pandas as pd
import numpy as np
import seaborn as sb

data = pd.read_pickle('Binning.pickle')

data

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot,JOININGYEAR_BIN,AGE_BIN,EXPERIENCEINCURRENTDOMAIN_BIN
0,Bachelors,2017,A,3,34,Male,No,0,0,"(2016.0, 2017.0]","(31.0, 34.0]","(-0.001, 1.0]"
1,Bachelors,2013,C,1,28,Female,No,3,1,"(2011.999, 2013.0]","(27.0, 28.0]","(2.0, 3.0]"
2,Bachelors,2014,B,3,38,Female,No,2,0,"(2013.0, 2014.0]","(34.0, 38.0]","(1.0, 2.0]"
4,Masters,2017,C,3,24,Male,Yes,2,1,"(2016.0, 2017.0]","(21.999, 25.0]","(1.0, 2.0]"
5,Bachelors,2016,A,3,22,Male,No,0,0,"(2015.0, 2016.0]","(21.999, 25.0]","(-0.001, 1.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...
4645,Masters,2017,C,2,31,Female,No,2,0,"(2016.0, 2017.0]","(29.0, 31.0]","(1.0, 2.0]"
4646,Bachelors,2013,A,3,25,Female,No,3,0,"(2011.999, 2013.0]","(21.999, 25.0]","(2.0, 3.0]"
4648,Bachelors,2013,A,3,26,Female,No,4,0,"(2011.999, 2013.0]","(25.0, 26.0]","(3.0, 4.0]"
4651,Bachelors,2012,A,3,30,Male,Yes,2,0,"(2011.999, 2013.0]","(29.0, 31.0]","(1.0, 2.0]"


In [13]:
def num_event(event, dataframe ,field, val):
    if event == 'good':
        event = 1
    else:
        event = 0
    cond_main = dataframe[field] == val
    cond_event = dataframe['LeaveOrNot'] == event
    n = len(dataframe[cond_main & cond_event].index)
    return n


def dist_event(event, dataframe ,field, val):
    n = num_event(event, dataframe, field, val)
    sumd = 0
    all_elements = set(dataframe[field].to_numpy())
    for a in all_elements:
        s = num_event(event, dataframe, field, a)
        sumd = sumd + s
    return n/sumd

def WOE(dataframe, field, val):
    dg = dist_event('good', dataframe, field, val)
    db = dist_event('bad', dataframe, field, val)
    w = np.log(dg / db)
    return w
dt_train = pd.DataFrame()
dt_train['LeaveOrNot'] = data['LeaveOrNot']
ATTRS = ['Education', 'PaymentTier', 'Age', 'Gender','JOININGYEAR_BIN']


NEW_WOE_ATTRS=[]
for attr in ATTRS:
    attr_element = set(data[attr].to_numpy())
    attr_woe = dict()
    for i in attr_element:
        w = WOE(data, attr, i)
        attr_woe[i] = w
        NEW_WOE_ATTRS.append( attr_woe[i])
    print(f'WOE {attr} -> {attr_woe}')
    

    lst = []
    for s in data[attr]:
        woe = attr_woe[s]
        lst.append(woe)

    new_field = attr + '_WOE'
    dt_train[new_field] = lst


dt_train


WOE Education -> {'PHD': -0.28301193536096, 'Masters': 0.6000994798579106, 'Bachelors': -0.1451637740319534}
WOE PaymentTier -> {1: 0.09518414463165278, 2: 1.1020621693537014, 3: -0.33065084261457534}
WOE Age -> {22: -0.06146966541372413, 23: 0.13350300893378958, 24: 0.1959384798745817, 25: 0.32902607758628005, 26: 0.01524525216713775, 27: 0.05749053341567421, 28: -0.25032719318746427, 29: -0.0012295850363674451, 30: 0.11993427972772051, 31: -0.2517593918568555, 32: 0.32110162282858795, 33: -0.12243036520341118, 34: -0.19757837905254705, 35: 0.267034401558312, 36: 0.0663637060961608, 37: -0.4261127790016333, 38: -0.24379122220767863, 39: -0.3835531645828374, 40: -0.020647670893468917, 41: -0.3615742578640622}
WOE Gender -> {'Male': -0.4023102271788791, 'Female': 0.5209496115392753}
WOE JOININGYEAR_BIN -> {Interval(2011.999, 2013.0, closed='right'): -0.27966184847567804, Interval(2016.0, 2017.0, closed='right'): -0.3017918300125561, Interval(2015.0, 2016.0, closed='right'): -0.541764980

,LeaveOrNot,Education_WOE,PaymentTier_WOE,Age_WOE,Gender_WOE,JOININGYEAR_BIN_WOE
0,0,-0.145164,-0.330651,-0.197578,-0.40231,-0.301792
1,1,-0.145164,0.095184,-0.250327,0.52095,-0.279662
2,0,-0.145164,-0.330651,-0.243791,0.52095,-0.542618
4,1,0.600099,-0.330651,0.195938,-0.40231,-0.301792
5,0,-0.145164,-0.330651,-0.061470,-0.40231,-0.541765
...,...,...,...,...,...,...
4645,0,0.600099,1.102062,-0.251759,0.52095,-0.301792
4646,0,-0.145164,-0.330651,0.329026,0.52095,-0.279662
4648,0,-0.145164,-0.330651,0.015245,0.52095,-0.279662
4651,0,-0.145164,-0.330651,0.119934,-0.40231,-0.279662


In [14]:
Y =  dt_train['LeaveOrNot'].to_numpy()
x_1 = dt_train['Education_WOE'].to_numpy()
x_2 = dt_train['PaymentTier_WOE'].to_numpy()
x_3 = dt_train['Age_WOE'].to_numpy()
x_4 = dt_train['Gender_WOE'].to_numpy()
x_5 = dt_train['JOININGYEAR_BIN_WOE'].to_numpy()




array_ones = np.ones(x_1.shape)

X = np.array([array_ones, x_1,x_2,x_3,x_4,x_5])
X = np.transpose(X)
X

array([[ 1.        , -0.14516377, -0.33065084, -0.19757838, -0.40231023,
        -0.30179183],
       [ 1.        , -0.14516377,  0.09518414, -0.25032719,  0.52094961,
        -0.27966185],
       [ 1.        , -0.14516377, -0.33065084, -0.24379122,  0.52094961,
        -0.54261786],
       ...,
       [ 1.        , -0.14516377, -0.33065084,  0.01524525,  0.52094961,
        -0.27966185],
       [ 1.        , -0.14516377, -0.33065084,  0.11993428, -0.40231023,
        -0.27966185],
       [ 1.        , -0.14516377, -0.33065084, -0.12243037, -0.40231023,
         0.25264566]])

In [15]:
np.matmul(np.transpose(X), X)


array([[ 3.25800000e+03, -3.71923858e+01, -1.07448898e+02,
        -2.34016304e+01, -1.02179591e+02,  4.76645773e+02],
       [-3.71923858e+01,  2.82231695e+02,  1.41904051e+02,
         3.21090737e+00,  1.73863356e+01,  3.43534038e+01],
       [-1.07448898e+02,  1.41904051e+02,  1.02800071e+03,
         9.47633644e+00,  2.22375124e+02, -2.81442814e+02],
       [-2.34016304e+01,  3.21090737e+00,  9.47633644e+00,
         1.43520531e+02, -2.41235001e+00,  2.06777483e+01],
       [-1.02179591e+02,  1.73863356e+01,  2.22375124e+02,
        -2.41235001e+00,  6.70700052e+02, -4.82030429e+01],
       [ 4.76645773e+02,  3.43534038e+01, -2.81442814e+02,
         2.06777483e+01, -4.82030429e+01,  6.99739407e+03]])

In [16]:
np.linalg.inv(np.matmul(np.transpose(X), X))


array([[ 3.12465958e-04,  3.41025503e-05,  1.28654055e-05,
         5.30259797e-05,  4.11491029e-05, -2.08076300e-05],
       [ 3.41025503e-05,  3.82691038e-03, -5.54830097e-04,
        -3.58091324e-05,  8.67500480e-05, -4.27234919e-05],
       [ 1.28654055e-05, -5.54830097e-04,  1.14151573e-03,
        -7.34533571e-05, -3.59128402e-04,  4.55036870e-05],
       [ 5.30259797e-05, -3.58091324e-05, -7.34533571e-05,
         6.98673155e-03,  5.65751161e-05, -2.66470839e-05],
       [ 4.11491029e-05,  8.67500480e-05, -3.59128402e-04,
         5.65751161e-05,  1.61379134e-03, -6.72364682e-06],
       [-2.08076300e-05, -4.27234919e-05,  4.55036870e-05,
        -2.66470839e-05, -6.72364682e-06,  1.46400094e-04]])

In [17]:
np.matmul(np.transpose(X), Y)


array([1101.        ,   53.71213253,  204.77237999,   23.74452428,
        115.6286423 , 1088.05716091])

In [18]:
B = np.matmul(np.linalg.inv(np.matmul(np.transpose(X), X)), np.matmul(np.transpose(X), Y))
B

array([0.33186842, 0.09217946, 0.22435557, 0.18486177, 0.15705325,
       0.14199543])

In [19]:
def predict(x):
    y = B[0] + B[1]*x+ B[2]*x+ B[3]*x+ B[4]*x+ B[5]*x
    return y

eq = f'y = {B[0]:.3f}'
for i in range(1, len(B)):
    eq = eq + f'+ ({B[i]:.3f})x_{i}'
print(eq)

y = 0.332+ (0.092)x_1+ (0.224)x_2+ (0.185)x_3+ (0.157)x_4+ (0.142)x_5


In [20]:
data_test = pd.read_pickle('Data_test.pickle')

attr = pd.read_pickle('attr.pickle')
attr

,ATTR,VAL,N_GOOD,N_BAD,DIST_GOOD,DIST_BAD,WOE
0,Age,22,12,25,0.010899,0.011590,-0.061470
1,Age,23,14,24,0.012716,0.011127,0.133503
2,Age,24,95,153,0.086285,0.070932,0.195938
3,Age,25,122,172,0.110808,0.079740,0.329026
4,Age,26,156,301,0.141689,0.139546,0.015245
5,Age,27,153,283,0.138965,0.131201,0.057491
6,Age,28,122,307,0.110808,0.142327,-0.250327
7,Age,29,52,102,0.047230,0.047288,-0.001230
8,Age,30,61,106,0.055404,0.049142,0.119934
9,Age,31,25,63,0.022707,0.029207,-0.251759


In [21]:
# y_predict
data_test['y_predict'] = B[0] + B[1]*dt_train['Education_WOE'] + B[2]*dt_train['PaymentTier_WOE'] \
        + B[3]*dt_train['Age_WOE'] + B[4]*dt_train['Gender_WOE'] + B[5]*dt_train['JOININGYEAR_BIN_WOE'] 

data_test['P'] = 1/(1+np.exp(-data_test['y_predict']))

data_test['predict'] = data_test['P'].map(lambda x: 1 if x >= 0.5 else 0)
data_test

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot,y_predict,P,predict
3,Masters,2016,A,3,27,Male,No,5,1,NaN,NaN,0
8,Bachelors,2016,C,3,23,Male,No,1,0,NaN,NaN,0
10,Masters,2012,A,3,27,Male,No,5,1,NaN,NaN,0
12,Bachelors,2018,C,3,32,Male,Yes,5,1,NaN,NaN,0
17,Bachelors,2014,C,3,34,Male,No,4,0,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4639,Bachelors,2017,A,3,28,Male,No,0,1,NaN,NaN,0
4644,Bachelors,2015,C,3,32,Female,Yes,1,1,NaN,NaN,0
4647,Bachelors,2016,C,3,30,Male,No,2,0,NaN,NaN,0
4649,Masters,2013,C,2,37,Male,No,2,1,NaN,NaN,0


In [22]:
n_correct = (data_test['LeaveOrNot'] == data_test['predict']).sum()
print(n_correct)

accuracy = n_correct/1395
print('accuracy is')
print(accuracy)

896
accuracy is
0.6422939068100358
